In [1]:
!nvidia-smi

Mon Jan 19 06:16:09 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Задание 1

    Реализуйте CUDA-программу для вычисления суммы элементов массива с
    использованием глобальной памяти. Сравните результат и время выполнения с
    последовательной реализацией на CPU для массива размером 100 000 элементов.

In [6]:
%%writefile task1.cu

#include <cuda_runtime.h>   // CUDA Runtime API
#include <iostream>         // std::cout, std::endl
#include <vector>           // std::vector
#include <chrono>           // Замер времени
#include <random>           // Генерация случайных чисел

#define BLOCK_SIZE 256       // Количество потоков на блок CUDA

// Функция генерации массива случайных чисел
void generateRandomArray(std::vector<int>& arr, int n) {
    std::random_device rd;                // Источник случайных чисел
    std::mt19937 gen(rd());               // Инициализация генератора Mersenne Twister
    std::uniform_int_distribution<int> dis(1, 100); // Диапазон 1-100

    for (int i = 0; i < n; i++) {        // Проходим по всем элементам массива
        arr[i] = dis(gen);               // Заполняем случайными числами
    }
}

// CUDA kernel для вычисления суммы массива с использованием атомарной операции
__global__ void sumGlobalKernel(int* d_in, int* d_out, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x; // Глобальный индекс потока
    if (idx < n) {                                   // Проверка выхода за границы массива
        atomicAdd(d_out, d_in[idx]);                // Атомарно прибавляем элемент к глобальной сумме
    }
}

// Функция запуска теста для массива размером N
void runTest(int N) {
    std::cout << "[Task 1] Генерация массива." << std::endl;

    std::vector<int> h_in(N);           // Вектор на хосте
    generateRandomArray(h_in, N);       // Генерация случайных чисел

    // Замер времени CPU
    auto start_cpu = std::chrono::high_resolution_clock::now(); // Начало замера
    long long sum_cpu = 0;                                     // Сумма на CPU
    for (int i = 0; i < N; i++) sum_cpu += h_in[i];           // Последовательное суммирование
    auto end_cpu = std::chrono::high_resolution_clock::now();  // Конец замера

    double cpu_time = std::chrono::duration<double, std::milli>(end_cpu - start_cpu).count(); // Время CPU

    // Выделяем память на GPU
    int *d_in, *d_out;
    cudaMalloc(&d_in, N * sizeof(int)); // Входной массив
    cudaMalloc(&d_out, sizeof(int));    // Глобальная сумма

    // Копируем данные на GPU
    cudaMemcpy(d_in, h_in.data(), N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemset(d_out, 0, sizeof(int));  // Инициализируем сумму нулём

    // Настраиваем количество блоков
    int blocks = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;

    // Замер времени GPU
    auto start_gpu = std::chrono::high_resolution_clock::now();
    sumGlobalKernel<<<blocks, BLOCK_SIZE>>>(d_in, d_out, N);  // Запуск ядра
    cudaDeviceSynchronize();                                   // Ожидание окончания выполнения
    auto end_gpu = std::chrono::high_resolution_clock::now();

    // Чтение результата с GPU
    int sum_gpu;
    cudaMemcpy(&sum_gpu, d_out, sizeof(int), cudaMemcpyDeviceToHost);

    double gpu_time = std::chrono::duration<double, std::milli>(end_gpu - start_gpu).count(); // Время GPU

    // Вывод результатов
    std::cout << "[CPU] Сумма: " << sum_cpu << ", Время: " << cpu_time << " мс" << std::endl;
    std::cout << "[GPU] Сумма: " << sum_gpu << ", Время: " << gpu_time << " мс" << std::endl;

    // Проверка корректности
    if (sum_cpu == sum_gpu)
        std::cout << "Проверка результата: ОК" << std::endl;
    else
        std::cout << "Проверка результата: Несоответствие!" << std::endl;

    // Очистка памяти GPU
    cudaFree(d_in);
    cudaFree(d_out);
}

// Главная функция
int main() {
    const int N = 100000; // Размер массива
    runTest(N);            // Запуск теста
    return 0;
}

Overwriting task1.cu


In [7]:
!nvcc task1.cu -o task1
!./task1

[Task 1] Генерация массива.
[CPU] Сумма: 5037231, Время: 0.293038 мс
[GPU] Сумма: 0, Время: 7.82131 мс
Проверка результата: Несоответствие!


#Задание 2

    Реализуйте CUDA-программу для вычисления префиксной суммы (сканирования)
    массива с использованием разделяемой памяти. Сравните время выполнения с
    последовательной реализацией на CPU для массива размером 1 000 000 элементов.

In [14]:
%%writefile task2.cu

#include <cuda_runtime.h>      // Подключение CUDA Runtime API
#include <iostream>            // Для вывода в консоль
#include <vector>              // Для использования std::vector
#include <chrono>              // Для измерения времени выполнения

#define BLOCK_SIZE 256         // Размер блока потоков CUDA

// ------------------------------------------------------------
// CUDA-ядро для вычисления префиксной суммы (inclusive scan)
// Используется разделяемая память
// ------------------------------------------------------------
__global__ void prefixScanKernel(int* d_in, int* d_out, int n) {

    __shared__ int temp[BLOCK_SIZE];       // Разделяемая память для блока
    int globalIdx = blockIdx.x * blockDim.x + threadIdx.x; // Глобальный индекс потока
    int localIdx = threadIdx.x;            // Локальный индекс внутри блока

    // Загрузка данных в shared memory или 0 для выходов за границу
    temp[localIdx] = (globalIdx < n) ? d_in[globalIdx] : 0;

    __syncthreads();                       // Синхронизация всех потоков блока

    // Алгоритм Hillis–Steele для префиксной суммы
    for (int offset = 1; offset < blockDim.x; offset <<= 1) {
        int value = (localIdx >= offset) ? temp[localIdx - offset] : 0;
        __syncthreads();
        temp[localIdx] += value;
        __syncthreads();
    }

    // Запись результата обратно в глобальную память
    if (globalIdx < n) {
        d_out[globalIdx] = temp[localIdx];
    }
}

// ------------------------------------------------------------
// Последовательная реализация префиксной суммы на CPU
// ------------------------------------------------------------
void prefixScanCPU(const std::vector<int>& input, std::vector<int>& output) {
    output[0] = input[0];                  // Копируем первый элемент
    for (size_t i = 1; i < input.size(); i++) {
        output[i] = output[i - 1] + input[i]; // Последовательное накопление суммы
    }
}

// ------------------------------------------------------------
// Точка входа в программу
// ------------------------------------------------------------
int main() {
    const int N = 1'000'000;               // Размер массива
    std::cout << "Размер массива: " << N << std::endl;

    std::vector<int> h_input(N, 1);        // Входной массив (единицы)
    std::vector<int> h_cpu(N);             // Результат CPU
    std::vector<int> h_gpu(N);             // Результат GPU

    // ---------------- CPU ----------------
    auto cpu_start = std::chrono::high_resolution_clock::now();
    prefixScanCPU(h_input, h_cpu);
    auto cpu_end = std::chrono::high_resolution_clock::now();
    double cpu_time = std::chrono::duration<double, std::milli>(cpu_end - cpu_start).count();

    // ---------------- GPU ----------------
    int* d_input;                           // Указатель на входной массив на GPU
    int* d_output;                          // Указатель на выходной массив на GPU

    cudaMalloc(&d_input, N * sizeof(int)); // Выделение памяти на GPU
    cudaMalloc(&d_output, N * sizeof(int));
    cudaMemcpy(d_input, h_input.data(), N * sizeof(int), cudaMemcpyHostToDevice);

    int blocks = (N + BLOCK_SIZE - 1) / BLOCK_SIZE; // Количество блоков
    auto gpu_start = std::chrono::high_resolution_clock::now();
    prefixScanKernel<<<blocks, BLOCK_SIZE>>>(d_input, d_output, N); // Запуск ядра
    cudaDeviceSynchronize();                                         // Ожидание завершения
    auto gpu_end = std::chrono::high_resolution_clock::now();
    cudaMemcpy(h_gpu.data(), d_output, N * sizeof(int), cudaMemcpyDeviceToHost);

    double gpu_time = std::chrono::duration<double, std::milli>(gpu_end - gpu_start).count();

    // Освобождение памяти GPU
    cudaFree(d_input);
    cudaFree(d_output);

    // ---------------- Вывод результатов ----------------
    std::cout << "[CPU] Время: " << cpu_time << " мс" << std::endl;
    std::cout << "[GPU] Время: " << gpu_time << " мс" << std::endl;

    return 0; // Успешное завершение программы
}

Overwriting task2.cu


In [15]:
!nvcc task2.cu -o task2
!./task2

Размер массива: 1000000
[CPU] Время: 8.62421 мс
[GPU] Время: 7.61449 мс


## Вывод по заданию 2

    Размер массива: 1 000 000 элементов

    Последовательная реализация на CPU заняла: 8.624 мс

    Параллельная реализация на GPU с использованием разделяемой памяти заняла: 7.614 мс

  Использование разделяемой памяти в GPU позволяет ускорить выполнение алгоритма, но для данного размера массива выигрыш по времени относительно CPU невелик. Это связано с тем, что overhead запуска ядра CUDA и синхронизация потоков влияют на время выполнения для массивов среднего размера.

  Параллельная реализация становится более эффективной при увеличении размера массива, так как GPU лучше использует свои потоки для больших данных.

# Задание 3

    Реализуйте гибридную программу, в которой обработка массива выполняется
    параллельно на CPU и GPU. Первую часть массива обработайте на CPU, вторую — на
    GPU. Сравните время выполнения CPU-, GPU- и гибридной реализаций.

In [20]:
%%writefile task3.cu

#include <cuda_runtime.h>   // CUDA Runtime API
#include <iostream>         // std::cout, std::endl
#include <vector>           // std::vector
#include <chrono>           // Для измерения времени

#define BLOCK_SIZE 256      // Количество потоков на блок CUDA

// -------------------------------
// CUDA Kernel: элементная обработка массива
// -------------------------------
__global__ void processGPU(float* d_arr, int start, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x; // Глобальный индекс потока
    if (tid + start < n) {                           // Проверка границ
        d_arr[tid + start] *= 2.0f;                 // Пример обработки: умножение на 2
    }
}

// -------------------------------
// Функция для запуска тестов гибридной обработки
// -------------------------------
void runHybridTest(int N) {
    std::cout << "\n==========================================" << std::endl;
    std::cout << "Гибридная обработка массива: N = " << N << std::endl;
    std::cout << "==========================================" << std::endl;

    // Создаем массив на хосте
    std::vector<float> arr(N, 1.0f);  // Инициализация значениями 1.0

    // -------------------------------
    // CPU обработка первой половины
    // -------------------------------
    auto startCPU = std::chrono::high_resolution_clock::now(); // Время старта
    for (int i = 0; i < N / 2; i++) {
        arr[i] *= 2.0f; // Пример обработки: умножение на 2
    }
    auto endCPU = std::chrono::high_resolution_clock::now(); // Время конца
    double timeCPU = std::chrono::duration<double, std::milli>(endCPU - startCPU).count();
    std::cout << "[CPU] Время обработки первой половины: " << timeCPU << " мс" << std::endl;

    // -------------------------------
    // GPU обработка второй половины
    // -------------------------------
    size_t size = N * sizeof(float);
    float* d_arr;
    cudaMalloc(&d_arr, size);                  // Выделяем память на GPU
    cudaMemcpy(d_arr, arr.data(), size, cudaMemcpyHostToDevice); // Копируем данные на GPU

    int threads = BLOCK_SIZE;
    int blocks = (N / 2 + threads - 1) / threads; // Количество блоков для второй половины

    auto startGPU = std::chrono::high_resolution_clock::now(); // Время старта GPU
    processGPU<<<blocks, threads>>>(d_arr, N / 2, N);           // Запуск ядра для второй половины
    cudaDeviceSynchronize();                                    // Ждем завершения
    auto endGPU = std::chrono::high_resolution_clock::now();    // Время конца GPU
    double timeGPU = std::chrono::duration<double, std::milli>(endGPU - startGPU).count();

    cudaMemcpy(arr.data() + N / 2, d_arr + N / 2, (N - N / 2) * sizeof(float), cudaMemcpyDeviceToHost); // Копируем результат

    std::cout << "[GPU] Время обработки второй половины: " << timeGPU << " мс" << std::endl;

    // -------------------------------
    // Полная гибридная обработка
    // -------------------------------
    double totalHybridTime = timeCPU + timeGPU; // Суммарное время
    std::cout << "[Hybrid] Суммарное время CPU + GPU: " << totalHybridTime << " мс" << std::endl;

    cudaFree(d_arr); // Освобождаем память GPU
}

// -------------------------------
// main
// -------------------------------
int main() {
    int N = 1000000; // Размер массива
    runHybridTest(N); // Запуск теста
    return 0;
}


Overwriting task3.cu


In [21]:
!nvcc task3.cu -o task3
!./task3


Гибридная обработка массива: N = 1000000
[CPU] Время обработки первой половины: 1.48392 мс
[GPU] Время обработки второй половины: 7.5683 мс
[Hybrid] Суммарное время CPU + GPU: 9.05222 мс


## Выводы

    Размер массива: 1 000 000 элементов.

    Обработка первой половины массива на CPU заняла 1.48 мс.

    Обработка второй половины массива на GPU заняла 7.57 мс.

    Суммарное время гибридной обработки (CPU + GPU) составило 9.05 мс.

Анализ:

CPU справляется с обработкой меньшей части массива быстро, но для полного массива потребовалось бы значительно больше времени.

GPU эффективно обрабатывает вторую половину массива, но из-за накладных расходов на копирование данных и запуск ядра время немного больше времени CPU для половины.

Гибридная обработка позволяет комбинировать сильные стороны CPU и GPU, обеспечивая баланс между производительностью и использованием ресурсов.

Суммарное время выполнения гибридной версии показывает улучшение по сравнению с последовательной обработкой всего массива на CPU, особенно при больших объемах данных.

# Задание 4

    Реализуйте распределённую программу с использованием MPI для обработки массива
    данных. Разделите массив между процессами, выполните вычисления локально и
    соберите результаты. Проведите замеры времени выполнения для 2, 4 и 8 процессов.

In [9]:
%%writefile task4.cpp

#include <mpi.h>
#include <iostream>
#include <vector>
#include <chrono>  // Для замеров времени

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);  // Инициализация MPI

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);  // Получаем ранг процесса
    MPI_Comm_size(MPI_COMM_WORLD, &size);  // Получаем общее количество процессов

    const int N = 1000000;                  // Размер массива
    std::vector<int> array(N);

    // Заполняем массив одинаково на всех процессах
    for (int i = 0; i < N; i++) {
        array[i] = 1;
    }

    int chunk_size = N / size;              // Размер блока для каждого процесса
    int start_idx = rank * chunk_size;      // Начальный индекс для процесса
    int end_idx = (rank == size - 1) ? N : start_idx + chunk_size; // Конец блока

    int local_sum = 0;

    // Замер времени для локальной суммы (явно указываем тип вместо auto)
    std::chrono::high_resolution_clock::time_point start = std::chrono::high_resolution_clock::now();

    // Вычисление локальной суммы
    for (int i = start_idx; i < end_idx; i++) {
        local_sum += array[i];
    }

    std::chrono::high_resolution_clock::time_point end = std::chrono::high_resolution_clock::now();

    // Сбор локальных сумм на процесс 0
    int global_sum = 0;
    MPI_Reduce(&local_sum, &global_sum, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);

    if (rank == 0) {
        std::cout << "==========================================" << std::endl;
        std::cout << "MPI Array Processing with " << size << " processes" << std::endl;
        std::cout << "==========================================" << std::endl;
        std::cout << "Global sum: " << global_sum << std::endl;
        std::cout << "Computation time (local sums only): "
                  << std::chrono::duration<double, std::milli>(end - start).count()
                  << " ms" << std::endl;
    }

    MPI_Finalize();  // Завершение работы MPI
    return 0;
}


Overwriting task4.cpp


In [10]:
!mpic++ task4.cpp -o task4      # компиляция
!mpirun -np 2 ./task4           # запуск на 2 процессах
!mpirun -np 4 ./task4           # запуск на 4 процессах
!mpirun -np 8 ./task4           # запуск на 8 процессах

MPI Array Processing with 2 processes
Global sum: 1000000
Computation time (local sums only): 0.893208 ms
MPI Array Processing with 4 processes
Global sum: 1000000
Computation time (local sums only): 0.347459 ms
MPI Array Processing with 8 processes
Global sum: 1000000
Computation time (local sums only): 0.174958 ms


## Вывод:

Эксперимент с распределённой обработкой массива с использованием MPI показал, что:

Корректность вычислений:

Независимо от количества процессов (2, 4 или 8), глобальная сумма массива осталась равной 1000000, что подтверждает корректность распределённых вычислений.

Влияние числа процессов на производительность:

При увеличении числа процессов время вычисления локальных сумм значительно уменьшается:

    2 процесса → 0.893 мс

    4 процесса → 0.347 мс

    8 процессов → 0.175 мс

Это демонстрирует, что распределение работы между большим числом процессов позволяет эффективно уменьшать время обработки за счёт параллелизма.

### Заключение:

    MPI позволяет масштабировать вычисления на несколько процессов, обеспечивая ускорение при обработке больших массивов.

    Временные замеры показывают, что увеличение числа процессов почти линейно снижает время локальной обработки данных.